In [1]:
import pandas as pd
import nltk
import re
import numpy as np
import time 
import pickle
#nltk.download('punkt')


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn import metrics 
from sklearn.metrics import classification_report
import warnings 
warnings.filterwarnings("ignore")

#for RNN
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-04-07 14:12:40.753843: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [18]:
#import pre-processed data from pickle 
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Documents/GitHub/Masters-Thesis/sample_results_df_09.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2011_text_wo_names.pickle")
df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/2009_text_wo_names.pickle")
df_09

,pre_processed_sent,male_count,female_count,apicall_fail,sentences,article_id,year,col_type
0,"[break, silence, surround, break, say, band, l...",3,0,0,Liam Gallagher has broken the silence surround...,5048,2009,0
1,"[however, interview, say, longer]",2,0,0,"However, in an interview with The Times Liam G...",5048,2009,0
9,"[leave, band, follow, bust, say, simply, could...",4,0,0,Noel Gallagher left the Manchester band follow...,5048,2009,0
10,"[launch, clothing, line, earlier, year, admit,...",3,0,0,"""Liam launched his clothing line Pretty Green ...",5048,2009,0
13,"[people, able, buy, record]",1,0,0,"""People will be able to buy his records.",5048,2009,0
...,...,...,...,...,...,...,...,...
9706,"[organisers, say, extend, programme, live, tou...",2,0,0,Organisers of the X Factor have said they've e...,1043733,2009,0
9707,"[vote, judge, week, seven, competition]",1,0,0,Jedward were voted off by the X Factor judges ...,1043733,2009,0
9708,"[two, month, tour, begin, see, extra, date, add]",3,0,0,"The two month tour, which begins in Liverpool ...",1043733,2009,0
9709,"[artists, confirm, tour, contestant]",2,1,0,Artists confirmed for the tour are contestants...,1043733,2009,0


In [19]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [21]:
df_09['col_type'] = df_09.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)
df_09= df_09[df_09["col_type"].notnull()]
df_09

,pre_processed_sent,male_count,female_count,apicall_fail,sentences,article_id,year,col_type
0,"[break, silence, surround, break, say, band, l...",3,0,0,Liam Gallagher has broken the silence surround...,5048,2009,0
1,"[however, interview, say, longer]",2,0,0,"However, in an interview with The Times Liam G...",5048,2009,0
9,"[leave, band, follow, bust, say, simply, could...",4,0,0,Noel Gallagher left the Manchester band follow...,5048,2009,0
10,"[launch, clothing, line, earlier, year, admit,...",3,0,0,"""Liam launched his clothing line Pretty Green ...",5048,2009,0
13,"[people, able, buy, record]",1,0,0,"""People will be able to buy his records.",5048,2009,0
...,...,...,...,...,...,...,...,...
9701,"[say, remember, child, adult, massive, differe...",0,1,0,"She said: ""You have to remember that they are ...",1041754,2009,1
9705,"[say, sexual, offence, act, mean, sexual, rela...",0,1,0,"She said: ""The sexual offences act of 2003 mea...",1041754,2009,1
9706,"[organisers, say, extend, programme, live, tou...",2,0,0,Organisers of the X Factor have said they've e...,1043733,2009,0
9707,"[vote, judge, week, seven, competition]",1,0,0,Jedward were voted off by the X Factor judges ...,1043733,2009,0


**Define TFIDF Vectorizer**

In [23]:
#tfidf vectorizer
def fake(token):
    return token

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=fake,
    preprocessor=fake,
    token_pattern=None)  

**Word2Vec for Word Embedding**

In [3]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# Train a Word2Vec model on tokenized sentences
model_w2v = Word2Vec(df_09["pre_processed_sent"], size=100, window=5, min_count=1, workers=4)

# Convert the sentences into feature vectors using the Word2Vec model
X = np.array([np.mean([model_w2v.wv[word] for word in sentence], axis=0) for sentence in df_09["pre_processed_sent"]])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, df_09["col_type"], test_size=0.2, random_state=42)


TypeError: __init__() got an unexpected keyword argument 'size'

**Logistic Regression Classifier**

*What the LR model does-* LR estimates the probability of an instance belonging to the positive class. 

In [7]:
#word2vec to dos
import gensim
from gensim.models import Word2Vec
file_w2vec= (r"/Users/yolandaferreirofranchi/Desktop/GoogleNews-vectors-negative300.bin") #yolanda's path 
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(file_w2vec, binary=True)

In [24]:
def logistic_regression_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
        'solver': ['liblinear', 'lbfgs', 'newton-cg', 'sag' 'saga'], 
        'class_weight': ['balanced', {0: 0.3, 1: 0.7}],
        'random_state': [42]
    }

    #the classifier 
    clf = LogisticRegression()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = LogisticRegression(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #coefficients
    coefs = best_clf.coef_[0]
    sorted_coef = sorted((zip(tfidf.get_feature_names_out(), coefs)), key = lambda x: x[1], reverse=True)
    high_coef = sorted_coef[:100]
    low_coef = sorted_coef[-100:]
    
    df_high_coef = pd.DataFrame(high_coef, columns=['feature', 'coef'])
    df_low_coef = pd.DataFrame(low_coef, columns=['feature', 'coef'])


    #print probability results 
    feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    positive_probas = probas[:, 1]

    print(f"\nProbability of the Following Words Being Female:")
    feature_prob_dict = {}
    for feature, index in feature_indices.items():
        proba = (X_transformed[:, index].toarray() * positive_probas).mean()
        feature_prob_dict[feature] = proba
        print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return df_high_coef, df_low_coef

In [13]:
logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')

Accuracy: 0.91
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.94      0.93       683
           1       0.89      0.88      0.88       420

    accuracy                           0.91      1103
   macro avg       0.91      0.91      0.91      1103
weighted avg       0.91      0.91      0.91      1103


Probability of the Following Words Being Female:
abandon: 0.00032256995801017204
abdelbaset: 4.383154332351764e-05
abdomen: 7.068125729751178e-05
ability: 7.632170307152125e-05
able: 0.0008719819744825204
absence: 0.00012129829695041015
absolute: 3.27825452783881e-05
absolutely: 0.0004925375407898479
absolve: 5.3668359093339775e-05
absoultely: 5.040597328311759e-05
abuse: 0.000600818413815
abusive: 4.713754932244141e-05
accept: 0.00026746598483805647
acceptance: 3.983958019506171e-05
accepting: 5.259797608872118e-05
access: 7.18515208766094e-05
accessible: 8.145316784597065e-05
accident: 9.879044727267164e-05
accidental: 6.064

(     feature       coef
 0   goulding  47.991392
 1    firearm  43.471260
 2         cv  42.131377
 3      lewis  41.393826
 4      elbow  38.898314
 ..       ...        ...
 95     agent  23.222024
 96  investor  22.839873
 97      fair  22.833241
 98   entitle  22.807738
 99     giant  22.783218
 
 [100 rows x 2 columns],
           feature       coef
 0    improvements -16.739040
 1         provide -16.881251
 2   international -17.067813
 3            mgmt -17.103490
 4         opening -17.211585
 ..            ...        ...
 95         arrest -44.270663
 96         bottom -46.092269
 97           bank -46.294924
 98        country -52.630831
 99           fuel -57.070521
 
 [100 rows x 2 columns])

**Coefficient Analysis** 

In [25]:
#call the function - with accuracy, classification, highest & lowest coeffs, df with word probs
df_coef_09 = logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')
#df_coef_09

Accuracy: 0.65
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       405
           1       0.51      0.52      0.52       221

    accuracy                           0.65       626
   macro avg       0.62      0.62      0.62       626
weighted avg       0.65      0.65      0.65       626


Probability of the Following Words Being Female:
abandon: 0.0003404661284511914
abc: 0.0001268037024366298
abdomen: 6.533978376986191e-05
abiding: 4.665482025045693e-05
ability: 7.049183075621901e-05
able: 0.000723353898705268
aboard: 3.3806153624663705e-05
abound: 3.691751804628137e-05
absence: 0.00020210207052996895
absolutely: 0.0002663821382460407
absoultely: 4.644004587416058e-05
abuse: 0.0006204412117592739
abusive: 8.701281779890703e-05
academy: 8.60830573020963e-05
accept: 0.00029064956327307486
acceptance: 3.603439751725694e-05
accepting: 4.886123999862347e-05
access: 0.0001819801459824941
accident: 9.145331457750732e

*Interpreting Performance*

The LR model is WAY better in terms of precision, recall, and f1-score at predicting the negative class - i.e. male. 

In [17]:
#create DF of highest coef
highest_coef_09 = pd.DataFrame(df_coef_09[0]) 

#create DF of lowest lowest coef manipulation 
lowest_coef_09 = pd.DataFrame(df_coef_09[1]) 
lowest_coef_09 = lowest_coef_09.sort_values(by = ["coef"], ascending = True).reset_index(drop = True) #absolute lowest value 
lowest_coef_09.head(20)

,feature,coef
0,fuel,-57.070521
1,country,-52.630831
2,bank,-46.294924
3,bottom,-46.092269
4,arrest,-44.270663
5,upcoming,-42.647138
6,commit,-40.943675
7,dog,-39.611463
8,blues,-38.088271
9,president,-37.909751


In [22]:
#save DF as pickle file per year 
lowest_coef_09.to_pickle('RESULTS09_coef_low.pickle')
highest_coef_09.to_pickle('RESULTS09_coef_high.pickle')

**Interpretation of coefficient results**
"winner" is one of the independent variables in the model and its coefficient value is 1.2865737872946597. This means that a one unit increase in the value of the "winner" variable will increase the log-odds of the positive class (e.g. "female" if the logistic regression model is binary and predicting gender) by the corresponding coefficient value, while holding all other variables constant.

**Word Probability Analysis** 

#make sure to un-comment the last two lines of the functions

In [10]:
#call the function - with accuracy, classification, highest & lowest coeffs, df with word probs
df_probs_09 = logistic_regression_year(df_09, 'col_type', 'pre_processed_sent')
df_probs_09

Accuracy: 0.66
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.71      0.73       418
           1       0.55      0.58      0.56       248

    accuracy                           0.66       666
   macro avg       0.64      0.65      0.64       666
weighted avg       0.67      0.66      0.67       666


Probability of the Following Words Being Female:
abandon: 0.00038120850269269033
abc: 0.0001413201190697638
abdomen: 7.339213954914968e-05
abiding: 5.2375749347855515e-05
ability: 7.871855922404075e-05
able: 0.0008132952952532201
aboard: 3.773704745159681e-05
abound: 4.120399451172335e-05
absence: 0.0002264022740987104
absolutely: 0.00035186146183954124
absoultely: 5.1772073732715285e-05
abuse: 0.0006958814132373457
abusive: 9.764637581888604e-05
academy: 9.623521697652816e-05
accept: 0.0003250303697333194
acceptance: 4.028711363185348e-05
accepting: 5.44636593844929e-05
access: 0.00020352834725587243
accessible: 4.33494228737

KeyboardInterrupt: 

In [ ]:
topwords_09 = df_probs_09["probability"].sort_values(ascending= False)

#top 100 words with highest probability of belonging to the female class 
topwords_09 = pd.DataFrame(topwords_09) 
topwords_09.to_pickle('RESULTS09_sample.pickle')
topwords_09.head(100)

**Interpretation of the predicted probabilities in LR:** 
The predicted probabilities of the logistic regression model tell us the probability that the input data belongs to the positive class - in this case the female class as we attributed it a value = 1 in binary log reg. Hence, for each word, we get a list of a word/feature and the probability that it is female. 

These predicted probabilities can be interpreted as the confidence level of the model in its prediction. For example, a predicted probability of 0.8 for a positive class means that the model is 80% confident that the sample belongs to the positive class. 

**Support Vector Machine**

In [19]:
from sklearn.svm import SVC
from sklearn.inspection import permutation_importance
def svm_year(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    #train test split
    tfidf = TfidfVectorizer()
    X_transformed = tfidf.fit_transform(X).toarray()
    X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=42)

    # define the hyperparameters to search over
    param_grid = {
        'C': [.001, .01, .1, 1, 10, 100],
        #'kernel': default('rbf')',
        'degree': [2,3,4],
        'gamma': ['scale', 'auto'],
        #'class_weight': [None],
        #'random_state': [42]
    }

    #the classifier 
    clf = SVC()

    #create a GridsearchCV object 
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)

    best_clf = SVC(**grid_search.best_params_)

    #run the classifier 
    best_clf.fit(X_train, y_train)
    y_pred = best_clf.predict(X_test)

    #performance 
    accuracy = best_clf.score(X_test, y_test) #evaluate on test set
    class_report = classification_report(y_test, y_pred, zero_division = 0)
    results = {'accuracy': accuracy, 'classification_report': class_report}
    #print(f"Year: {year}")
    print(f"Accuracy: {accuracy:.2f}")
    print(f"Classification Report:\n{class_report}")

    #get important features (equivalent of coefficients in logreg)
    result_clf = permutation_importance(best_clf, X_test, y_test, n_repeats=10, random_state=42)
    importance_scores = result_clf.importances_mean
    importance_scores_df = pd.DataFrame(importance_scores, columns=['tbd'])

    for i in range(len(importance_scores)):
        print("Feature {}: Importance score = {:.3f}".format(i, importance_scores[i]))
    
    #print probability results 
    #feature_indices = {feature: idx for idx, feature in enumerate(tfidf.get_feature_names_out())}
    #probas = best_clf.predict_proba(X_transformed)  # predict probabilities of positive class
    #positive_probas = probas[:, 1]

    #print(f"\nProbability of the Following Words Being Female:")
    #feature_prob_dict = {}
    #for feature, index in feature_indices.items():
        #proba = (X_transformed[:, index] * positive_probas).mean()
        #feature_prob_dict[feature] = proba
        #print(f"{feature}: {proba}")
    
    #create a dataframe with the data: 
    #df_probs = pd.DataFrame.from_dict(feature_prob_dict, orient='index', columns=['probability'])
    
    #return df_probs #DF of probability for each word being female by year

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

    return importance_scores_df

In [20]:
svm_year(df_09, 'col_type', 'pre_processed_sent') #takes more than 1 hour to run

Accuracy: 0.70
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.88      0.78       418
           1       0.66      0.38      0.48       248

    accuracy                           0.70       666
   macro avg       0.68      0.63      0.63       666
weighted avg       0.69      0.70      0.67       666



KeyboardInterrupt: 

In [ ]:
#save the SVM model 
import joblib
model, accuracy, class_report = svm_year(df_09, 'col_type', 'pre_processed_sent')

# Save the model and performance metrics
joblib.dump(model, 'model_svm.pkl')
with open('performance_metrics.txt', 'w') as f:
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Classification Report:\n{class_report}")

**Neural Network** 

In [30]:
def RNN_lstm(df, target_col, text_col):
    #start timer 
    start_time = time.time()
    
    #split data 
    X = df[text_col].apply(lambda x: str(x))
    y = df[target_col]

    # Preprocess text
    tokenizer = Tokenizer(num_words=5000, oov_token='<OOV>')
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    padded_sequences = pad_sequences(sequences, maxlen=100, truncating='post', padding='post')
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(padded_sequences, y, test_size=0.2, random_state=42)
    
    # Define model
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(10000, 32),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))
    
    # Evaluate model on test set
    loss, accuracy = model.evaluate(X_test, y_test)
    print('Test loss:', loss)
    print('Test accuracy:', accuracy)

    #end timer 
    end_time = time.time()
    print(f"\nExecution time: {end_time - start_time:.2f} seconds")

In [31]:
RNN_lstm(df_09, 'col_type', 'pre_processed_sent')

Epoch 1/10


2023-04-06 18:18:08.183364: W tensorflow/core/framework/op_kernel.cc:1757] OP_REQUIRES failed at cast_op.cc:121 : UNIMPLEMENTED: Cast string to float is not supported


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 711, in start
      self.io_loop.start()
    File "/Users/yolandaferreirofranchi/opt/anaconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py

**Comparing the Models**

In [25]:
model_results = {' ': ['Logistic Regression', 'SVM', 'NN'],
           'Accuracy': [0.66, 0.70, 0],
           'Time (s)': [54.39, 3540, 0]}

# Create a pandas dataframe from the dictionary
df_mod_results = pd.DataFrame(model_results)

# Set the index of the dataframe to the Kernel column
df_mod_results.set_index(' ', inplace=True)

# Display the dataframe
print(df_mod_results)

                     Accuracy  Time (s)
                                       
Logistic Regression      0.66     54.39
SVM                      0.70   3540.00
NN                       0.00      0.00
